In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# python_version 3.7.10

### &nbsp;&nbsp;&nbsp; %CLASS% StockClass: used to create the stock dictionary

In [2]:
class StockClass(object):

    # Initialization of the StockClass object with the ticker symbol which is use to construct a yf.Ticker object
    def __init__(self, ticker, isin=None, exchangeid=None, sector=None, industry=None, country=None, pe=None, eps=None,
                 insiderown=None, shsout=None, shsfloat=None, mktcap=None, income=None, sales=None,
                 booksh=None, pb=None, roa=None, tp=None, roe=None, roi=None, employees=None, debteq=None, 
                 groupby=None, confusion_matrix=None, accuracy_report=None, 
                 confusion_matrix2=None, accuracy_report2=None):
        self.name = ticker
        self.isin = isin
        self.exchangeid = exchangeid
        # self.history is the method of the SotckClass object to store data in DataFrame format
        self.history = pd.DataFrame
        self.investing = pd.DataFrame
        self.pickle = pd.DataFrame
        self.not_found = np.array([['date', 'ticker']])
        self.nanDiv = False
        self.nanSplit = False
        self.sector = None
        self.industry = industry
        self.country = country
        self.pe = pe
        self.eps = eps
        self.insiderown = insiderown
        self.shsout = shsout
        self.shsfloat = shsfloat
        self.mktcap = mktcap
        self.income = income
        self.sales = sales
        self.bookh = booksh
        self.pb = pb
        self.roa = roa
        self.tp = tp
        self.roe = roe
        self.roi = roi
        self.employees = employees
        self.debteq = debteq
        self.rsi = pd.DataFrame
        self.groupby = groupby
        self.confusion_matrix = confusion_matrix
        self.accuracy_report = accuracy_report
        self.confusion_matrix2 = confusion_matrix2
        self.accuracy_report2 = accuracy_report2

In [3]:
class IndexClass(object):

    # Initialization of the IndexClass object
    def __init__(self, description='', groupby=None, open=None, high=None, low=None, adjclose=None):
        self.description = description
        self.groupby = groupby
        self.open =  pd.DataFrame
        self.high =  pd.DataFrame
        self.low =  pd.DataFrame
        self.adjclose =  pd.DataFrame

# 1. First part: import data, cleaning and arranging 

### &nbsp;&nbsp;&nbsp; * Main packages import

In [4]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

### &nbsp;&nbsp;&nbsp; * Data import

In [5]:
open_df_original = pd.read_csv('/content/drive/MyDrive/Data/20210322/open', index_col='date')
high_df_original = pd.read_csv('/content/drive/MyDrive/Data/20210322/high', index_col='date')
low_df_original = pd.read_csv('/content/drive/MyDrive/Data/20210322/low', index_col='date')
adjclose_df_original = pd.read_csv('/content/drive/MyDrive/Data/20210322/adjclose', index_col='date')
volume_df_original = pd.read_csv('/content/drive/MyDrive/Data/20210322/volume', index_col='date')

In [60]:
financials_df_original = pd.read_excel('/content/drive/MyDrive/Data/20210322/ticker_isin_file_financials.xlsx', na_values=['NA'], usecols = 'A:E,J', index_col='ticker')

In [61]:
ticker_list = list(adjclose_df_original.columns)

In [62]:
# to drop columns with the end of the series NaN (probably not quoted anymore)
if open_df_original.isnull().values.any():
    print('open shape before: ', open_df_original.shape)
    open_df_original.dropna(axis=1, how='any', inplace=True)
print('open shape after: ', open_df_original.shape)    
if high_df_original.isnull().values.any():
    print('high shape before: ', high_df_original.shape)
    high_df_original.dropna(axis=1, how='any', inplace=True)
print('high shape after: ', high_df_original.shape)  
if low_df_original.isnull().values.any():
    print('low shape before: ', low_df_original.shape)
    low_df_original.dropna(axis=1, how='any', inplace=True)
print('low shape after: ', low_df_original.shape)    
if adjclose_df_original.isnull().values.any():
    print('high shape before: ', adjclose_df_original.shape)
    adjclose_df_original.dropna(axis=1, how='any', inplace=True)
print('high shape after: ', adjclose_df_original.shape)    
if volume_df_original.isnull().values.any():
    print('high shape before: ', volume_df_original.shape)
    volume_df_original.dropna(axis=1, how='any', inplace=True)
print('high shape after: ', volume_df_original.shape)   
if financials_df_original.isnull().values.any():
    print('financials shape before: ', financials_df_original.shape)
    financials_df_original.dropna(axis=0, how='any', inplace=True)
print('financials shape after: ', financials_df_original.transpose().shape)      

open shape after:  (415, 4534)
high shape after:  (415, 4534)
low shape after:  (415, 4534)
high shape after:  (415, 4534)
high shape after:  (415, 4534)
financials shape before:  (5143, 5)
financials shape after:  (5, 4731)


In [63]:
open_df = open_df_original.drop(index=open_df_original.iloc[-1:].index)
high_df = high_df_original.drop(index=high_df_original.iloc[-1:].index)
low_df = low_df_original.drop(index=low_df_original.iloc[-1:].index)
adjclose_df = adjclose_df_original.drop(index=adjclose_df_original.iloc[-1:].index)
volume_df = volume_df_original.drop(index=volume_df_original.iloc[-1:].index)

# 2 Second part: computation and assessing 

# 2.a Indeces computation

### &nbsp;&nbsp;&nbsp; * Definition of the StockClass dictionary

In [64]:
# Definition of a dictionary to store stock as StockClass instances and for each stock get attribute
# (take a look at _2_0_stock_dataframe_class.py for more information)
stock_object_dictionary = {'{0}'.format(ticker): StockClass(ticker=ticker_list) for ticker in ticker_list}

In [65]:
financials_df_original.rename(columns={'Shs Outstand':'market_cap', 'Sector':'sector', 'Industry':'industry'}, inplace=True)
financials_df = financials_df_original.dropna(axis=0, how='any')
financials_df = financials_df[financials_df['market_cap']!='-']

In [66]:
# Remove rows which has no market data in price dataframe
main_list = list(set(financials_df.index) - set(adjclose_df.columns))
financials_df.drop(index=main_list, inplace=True)

In [67]:
# GroupBy
exchange_id_group = financials_df.groupby(['exchange_id'])
sector_group = financials_df.groupby(['sector'])
industry_group = financials_df.groupby(['industry'])

num_of_quantiles = 10
financials_df['market_cap_group'] = np.nan
mkt_cap_quantiles = []
for q in np.linspace(1/num_of_quantiles, 1, num_of_quantiles):
  mkt_cap_quantiles.append(financials_df['market_cap'].quantile(q))
for ticker in financials_df.index:
    for i, q in enumerate(mkt_cap_quantiles):
        if financials_df['market_cap'].loc[ticker] < q:
            financials_df['market_cap_group'].loc[ticker] = i
            break

# Group by Market Cap Range
marketcap_group = financials_df.groupby(['market_cap_group'])

In [68]:
# Definition of a dictionary to store indeces as IndexClass instances (exchange_id)
index_object_dictionary_exid = {'{0}'.format(exid): IndexClass(description=exid, groupby='exchange_id') for exid in exchange_id_group.groups.keys()}
# Definition of a dictionary to store indeces as IndexClass instances (Sector)
index_object_dictionary_sec = {'{0}'.format(sec): IndexClass(description=sec, groupby='sector') for sec in sector_group.groups.keys()}
#  Definition of a dictionary to store indeces as IndexClass instances (Industry)
index_object_dictionary_ind = {'{0}'.format(ind): IndexClass(description=ind, groupby='industry') for ind in industry_group.groups.keys()}
#  Definition of a dictionary to store indeces as IndexClass instances (Market Cap)
index_object_dictionary_mktcap = {'{0}'.format(mktcap): IndexClass(description=mktcap, groupby='market_cap') for mktcap in marketcap_group.groups.keys()}

In [69]:
forfiles = ['open', 'high', 'low', 'adjclose']

In [70]:
list_of_stock_to_erase = [] 
for i, exid in enumerate(exchange_id_group.groups.keys()):
  open_df_exchange_id = open_df[exchange_id_group.groups[list(exchange_id_group.groups.keys())[i]]]
  high_df_exchange_id = high_df[exchange_id_group.groups[list(exchange_id_group.groups.keys())[i]]]
  low_df_exchange_id = low_df[exchange_id_group.groups[list(exchange_id_group.groups.keys())[i]]]
  adjclose_df_exchange_id = adjclose_df[exchange_id_group.groups[list(exchange_id_group.groups.keys())[i]]]
  for ticker in exchange_id_group.groups[list(exchange_id_group.groups.keys())[i]]:
    try:
      open_df_exchange_id[ticker] = open_df_exchange_id[ticker] * financials_df['market_cap'].loc[ticker]
      high_df_exchange_id[ticker] = high_df_exchange_id[ticker] * financials_df['market_cap'].loc[ticker]
      low_df_exchange_id[ticker] = low_df_exchange_id[ticker] * financials_df['market_cap'].loc[ticker]
      adjclose_df_exchange_id[ticker] = adjclose_df_exchange_id[ticker] * financials_df['market_cap'].loc[ticker]
    except:
      list_of_stock_to_erase.append(ticker)

  index_object_dictionary_exid['{0}'.format(exid)].open = open_df_exchange_id.sum(axis=1)/1000000000000000
  index_object_dictionary_exid['{0}'.format(exid)].high = high_df_exchange_id.sum(axis=1)/1000000000000000
  index_object_dictionary_exid['{0}'.format(exid)].low = low_df_exchange_id.sum(axis=1)/1000000000000000
  index_object_dictionary_exid['{0}'.format(exid)].adjclose = adjclose_df_exchange_id.sum(axis=1)/1000000000000000
  
for i, exid in enumerate(index_object_dictionary_exid.keys()):
  if i == 0:
    open = pd.DataFrame(index_object_dictionary_exid['{0}'.format(exid)].open, columns=[exid])
    high = pd.DataFrame(index_object_dictionary_exid['{0}'.format(exid)].high, columns=[exid])
    low = pd.DataFrame(index_object_dictionary_exid['{0}'.format(exid)].low, columns=[exid])
    adjclose = pd.DataFrame(index_object_dictionary_exid['{0}'.format(exid)].adjclose, columns=[exid])
  else:
    open = pd.concat([open, pd.DataFrame(index_object_dictionary_exid['{0}'.format(exid)].open, columns=[exid])], axis=1)
    high = pd.concat([high, pd.DataFrame(index_object_dictionary_exid['{0}'.format(exid)].high, columns=[exid])], axis=1)
    low = pd.concat([low, pd.DataFrame(index_object_dictionary_exid['{0}'.format(exid)].low, columns=[exid])], axis=1)
    adjclose = pd.concat([adjclose, pd.DataFrame(index_object_dictionary_exid['{0}'.format(exid)].adjclose, columns=[exid])], axis=1)

for f in forfiles:
  filename = 'exchange_id ' + f 
  if f == 'open':
    open.to_csv(filename, index=True)
  if f == 'high':
    high.to_csv(filename, index=True)
  if f == 'low':
    low.to_csv(filename, index=True)
  if f == 'adjclose':
    adjclose.to_csv(filename, index=True)

In [71]:
list_of_stock_to_erase = [] 
for i, sec in enumerate(sector_group.groups.keys()):
  open_df_sector = open_df[sector_group.groups[list(sector_group.groups.keys())[i]]]
  high_df_sector = high_df[sector_group.groups[list(sector_group.groups.keys())[i]]]
  low_df_sector = low_df[sector_group.groups[list(sector_group.groups.keys())[i]]]
  adjclose_df_sector = adjclose_df[sector_group.groups[list(sector_group.groups.keys())[i]]]
  for ticker in sector_group.groups[list(sector_group.groups.keys())[i]]:
    try:
      open_df_sector[ticker] = open_df_sector[ticker] * financials_df['market_cap'].loc[ticker]
      high_df_sector[ticker] = high_df_sector[ticker] * financials_df['market_cap'].loc[ticker]
      low_df_sector[ticker] = low_df_sector[ticker] * financials_df['market_cap'].loc[ticker]
      adjclose_df_sector[ticker] = adjclose_df_sector[ticker] * financials_df['market_cap'].loc[ticker]
    except:
      list_of_stock_to_erase.append(ticker)

  index_object_dictionary_sec['{0}'.format(sec)].open = open_df_sector.sum(axis=1)/1000000000000000
  index_object_dictionary_sec['{0}'.format(sec)].high = high_df_sector.sum(axis=1)/1000000000000000
  index_object_dictionary_sec['{0}'.format(sec)].low = low_df_sector.sum(axis=1)/1000000000000000
  index_object_dictionary_sec['{0}'.format(sec)].adjclose = adjclose_df_sector.sum(axis=1)/1000000000000000

for i, sec in enumerate(index_object_dictionary_sec.keys()):
  if i == 0:
    open = pd.DataFrame(index_object_dictionary_sec['{0}'.format(sec)].open, columns=[sec])
    high = pd.DataFrame(index_object_dictionary_sec['{0}'.format(sec)].high, columns=[sec])
    low = pd.DataFrame(index_object_dictionary_sec['{0}'.format(sec)].low, columns=[sec])
    adjclose = pd.DataFrame(index_object_dictionary_sec['{0}'.format(sec)].adjclose, columns=[sec])
  else:
    open = pd.concat([open, pd.DataFrame(index_object_dictionary_sec['{0}'.format(sec)].open, columns=[sec])], axis=1)
    high = pd.concat([high, pd.DataFrame(index_object_dictionary_sec['{0}'.format(sec)].high, columns=[sec])], axis=1)
    low = pd.concat([low, pd.DataFrame(index_object_dictionary_sec['{0}'.format(sec)].low, columns=[sec])], axis=1)
    adjclose = pd.concat([adjclose, pd.DataFrame(index_object_dictionary_sec['{0}'.format(sec)].adjclose, columns=[sec])], axis=1)

for f in forfiles:
  filename = 'sector ' + f 
  if f == 'open':
    open.to_csv(filename, index=True)
  if f == 'high':
    high.to_csv(filename, index=True)
  if f == 'low':
    low.to_csv(filename, index=True)
  if f == 'adjclose':
    adjclose.to_csv(filename, index=True)

In [72]:
list_of_stock_to_erase = [] 
for i, ind in enumerate(industry_group.groups.keys()):
  open_df_industry = open_df[industry_group.groups[list(industry_group.groups.keys())[i]]]
  high_df_industry = high_df[industry_group.groups[list(industry_group.groups.keys())[i]]]
  low_df_industry = low_df[industry_group.groups[list(industry_group.groups.keys())[i]]]
  adjclose_df_industry = adjclose_df[industry_group.groups[list(industry_group.groups.keys())[i]]]
  for ticker in industry_group.groups[list(industry_group.groups.keys())[i]]:
    try:
      open_df_industry[ticker] = open_df_industry[ticker] * financials_df['market_cap'].loc[ticker]
      high_df_industry[ticker] = high_df_industry[ticker] * financials_df['market_cap'].loc[ticker]
      low_df_industry[ticker] = low_df_industry[ticker] * financials_df['market_cap'].loc[ticker]
      adjclose_df_industry[ticker] = adjclose_df_industry[ticker] * financials_df['market_cap'].loc[ticker]
    except:
      list_of_stock_to_erase.append(ticker)

  index_object_dictionary_ind['{0}'.format(ind)].open = open_df_industry.sum(axis=1)/1000000000000000
  index_object_dictionary_ind['{0}'.format(ind)].high = high_df_industry.sum(axis=1)/1000000000000000
  index_object_dictionary_ind['{0}'.format(ind)].low = low_df_industry.sum(axis=1)/1000000000000000
  index_object_dictionary_ind['{0}'.format(ind)].adjclose = adjclose_df_industry.sum(axis=1)/1000000000000000

for i, ind in enumerate(index_object_dictionary_ind.keys()):
  if i == 0:
    open = pd.DataFrame(index_object_dictionary_ind['{0}'.format(ind)].open, columns=[ind])
    high = pd.DataFrame(index_object_dictionary_ind['{0}'.format(ind)].high, columns=[ind])
    low = pd.DataFrame(index_object_dictionary_ind['{0}'.format(ind)].low, columns=[ind])
    adjclose = pd.DataFrame(index_object_dictionary_ind['{0}'.format(ind)].adjclose, columns=[ind])
  else:
    open = pd.concat([open, pd.DataFrame(index_object_dictionary_ind['{0}'.format(ind)].open, columns=[ind])], axis=1)
    high = pd.concat([high, pd.DataFrame(index_object_dictionary_ind['{0}'.format(ind)].high, columns=[ind])], axis=1)
    low = pd.concat([low, pd.DataFrame(index_object_dictionary_ind['{0}'.format(ind)].low, columns=[ind])], axis=1)
    adjclose = pd.concat([adjclose, pd.DataFrame(index_object_dictionary_ind['{0}'.format(ind)].adjclose, columns=[ind])], axis=1)

for f in forfiles:
  filename = 'industry ' + f 
  if f == 'open':
    open.to_csv(filename, index=True)
  if f == 'high':
    high.to_csv(filename, index=True)
  if f == 'low':
    low.to_csv(filename, index=True)
  if f == 'adjclose':
    adjclose.to_csv(filename, index=True)

In [73]:
list_of_stock_to_erase = [] 
for i, mktcap in enumerate(marketcap_group.groups.keys()):
  open_df_marketcap = open_df[marketcap_group.groups[list(marketcap_group.groups.keys())[i]]]
  high_df_marketcap = high_df[marketcap_group.groups[list(marketcap_group.groups.keys())[i]]]
  low_df_marketcap = low_df[marketcap_group.groups[list(marketcap_group.groups.keys())[i]]]
  adjclose_df_marketcap = adjclose_df[marketcap_group.groups[list(marketcap_group.groups.keys())[i]]]
  for ticker in marketcap_group.groups[list(marketcap_group.groups.keys())[i]]:
    try:
      open_df_marketcap[ticker] = open_df_marketcap[ticker] * financials_df['market_cap'].loc[ticker]
      high_df_marketcap[ticker] = high_df_marketcap[ticker] * financials_df['market_cap'].loc[ticker]
      low_df_marketcap[ticker] = low_df_marketcap[ticker] * financials_df['market_cap'].loc[ticker]
      adjclose_df_marketcap[ticker] = adjclose_df_marketcap[ticker] * financials_df['market_cap'].loc[ticker]
    except:
      list_of_stock_to_erase.append(ticker)

  index_object_dictionary_mktcap['{0}'.format(mktcap)].open = open_df_marketcap.sum(axis=1)/1000000000000000
  index_object_dictionary_mktcap['{0}'.format(mktcap)].high = high_df_marketcap.sum(axis=1)/1000000000000000
  index_object_dictionary_mktcap['{0}'.format(mktcap)].low = low_df_marketcap.sum(axis=1)/1000000000000000
  index_object_dictionary_mktcap['{0}'.format(mktcap)].adjclose = adjclose_df_marketcap.sum(axis=1)/1000000000000000

for i, mktcap in enumerate(index_object_dictionary_mktcap.keys()):
  if i == 0:
    open = pd.DataFrame(index_object_dictionary_mktcap['{0}'.format(mktcap)].open, columns=[mktcap])
    high = pd.DataFrame(index_object_dictionary_mktcap['{0}'.format(mktcap)].high, columns=[mktcap])
    low = pd.DataFrame(index_object_dictionary_mktcap['{0}'.format(mktcap)].low, columns=[mktcap])
    adjclose = pd.DataFrame(index_object_dictionary_mktcap['{0}'.format(mktcap)].adjclose, columns=[mktcap])
  else:
    open = pd.concat([open, pd.DataFrame(index_object_dictionary_mktcap['{0}'.format(mktcap)].open, columns=[mktcap])], axis=1)
    high = pd.concat([high, pd.DataFrame(index_object_dictionary_mktcap['{0}'.format(mktcap)].high, columns=[mktcap])], axis=1)
    low = pd.concat([low, pd.DataFrame(index_object_dictionary_mktcap['{0}'.format(mktcap)].low, columns=[mktcap])], axis=1)
    adjclose = pd.concat([adjclose, pd.DataFrame(index_object_dictionary_mktcap['{0}'.format(mktcap)].adjclose, columns=[mktcap])], axis=1)

for f in forfiles:
  filename = 'market_cap ' + f 
  if f == 'open':
    open.to_csv(filename, index=True)
  if f == 'high':
    high.to_csv(filename, index=True)
  if f == 'low':
    low.to_csv(filename, index=True)
  if f == 'adjclose':
    adjclose.to_csv(filename, index=True)

In [87]:
list_of_history = ['price','exchange_id', 'sector', 'industry', 'market_cap'] 

In [89]:
# Definition of a dictionary to store OHLC of all indeces
indeces_object_dictionary = {'{0}'.format(ele): IndexClass(description=ele) for ele in list_of_history}

In [95]:
open_df_index.loc[str(financials_df[str(ele)].loc[ticker])]

KeyError: ignored

In [99]:
open_df_index

date,2019-07-29,2019-07-30,2019-07-31,2019-08-01,2019-08-02,2019-08-05,2019-08-06,2019-08-07,2019-08-08,2019-08-09,2019-08-12,2019-08-13,2019-08-14,2019-08-15,2019-08-16,2019-08-19,2019-08-20,2019-08-21,2019-08-22,2019-08-23,2019-08-26,2019-08-27,2019-08-28,2019-08-29,2019-08-30,2019-09-03,2019-09-04,2019-09-05,2019-09-06,2019-09-09,2019-09-10,2019-09-11,2019-09-12,2019-09-13,2019-09-16,2019-09-17,2019-09-18,2019-09-19,2019-09-20,2019-09-23,...,2021-01-21,2021-01-22,2021-01-25,2021-01-26,2021-01-27,2021-01-28,2021-01-29,2021-02-01,2021-02-02,2021-02-03,2021-02-04,2021-02-05,2021-02-08,2021-02-09,2021-02-10,2021-02-11,2021-02-12,2021-02-16,2021-02-17,2021-02-18,2021-02-19,2021-02-22,2021-02-23,2021-02-24,2021-02-25,2021-02-26,2021-03-01,2021-03-02,2021-03-03,2021-03-04,2021-03-05,2021-03-08,2021-03-09,2021-03-10,2021-03-11,2021-03-12,2021-03-15,2021-03-16,2021-03-17,2021-03-18
0.0,0.000605,0.000599,0.000601,0.000599,0.000587,0.000579,0.000578,0.000574,0.000579,0.000581,0.000576,0.000575,0.000572,0.000572,0.000578,0.000587,0.000580,0.000578,0.000580,0.000580,0.000575,0.000580,0.000574,0.000584,0.000590,0.000582,0.000586,0.000589,0.000591,0.000594,0.000598,0.000600,0.000609,0.000613,0.000611,0.000607,0.000604,0.000608,0.000605,0.000599,...,0.000679,0.000675,0.000673,0.000675,0.000666,0.000664,0.000666,0.000666,0.000673,0.000677,0.000681,0.000690,0.000691,0.000697,0.000710,0.000706,0.000702,0.000716,0.000716,0.000717,0.000717,0.000707,0.000717,0.000718,0.000728,0.000714,0.000720,0.000731,0.000728,0.000732,0.000723,0.000742,0.000753,0.000760,0.000775,0.000768,0.000766,0.000753,0.000744,0.000750
1.0,0.000220,0.000217,0.000220,0.000219,0.000215,0.000211,0.000209,0.000208,0.000212,0.000213,0.000212,0.000210,0.000210,0.000208,0.000208,0.000212,0.000212,0.000213,0.000213,0.000212,0.000208,0.000210,0.000207,0.000211,0.000213,0.000210,0.000210,0.000212,0.000214,0.000214,0.000215,0.000217,0.000219,0.000221,0.000221,0.000220,0.000220,0.000220,0.000219,0.000217,...,0.000228,0.000224,0.000228,0.000231,0.000225,0.000224,0.000224,0.000222,0.000227,0.000229,0.000231,0.000237,0.000240,0.000245,0.000248,0.000246,0.000245,0.000248,0.000245,0.000242,0.000242,0.000243,0.000241,0.000243,0.000246,0.000239,0.000240,0.000243,0.000241,0.000238,0.000236,0.000239,0.000244,0.000245,0.000249,0.000250,0.000252,0.000252,0.000248,0.000249
2.0,0.000353,0.000350,0.000354,0.000351,0.000344,0.000337,0.000335,0.000333,0.000337,0.000340,0.000337,0.000333,0.000334,0.000330,0.000331,0.000339,0.000338,0.000338,0.000338,0.000335,0.000330,0.000332,0.000326,0.000333,0.000336,0.000332,0.000333,0.000336,0.000340,0.000340,0.000340,0.000343,0.000348,0.000348,0.000347,0.000348,0.000350,0.000349,0.000348,0.000346,...,0.000473,0.000467,0.000474,0.000475,0.000463,0.000463,0.000459,0.000457,0.000469,0.000471,0.000475,0.000484,0.000490,0.000497,0.000503,0.000501,0.000496,0.000502,0.000493,0.000489,0.000491,0.000493,0.000486,0.000489,0.000495,0.000484,0.000490,0.000497,0.000491,0.000483,0.000475,0.000478,0.000486,0.000491,0.000499,0.000502,0.000506,0.000509,0.000502,0.000505
3.0,0.000510,0.000503,0.000508,0.000504,0.000497,0.000484,0.000481,0.000479,0.000487,0.000493,0.000487,0.000483,0.000481,0.000478,0.000479,0.000490,0.000487,0.000490,0.000492,0.000487,0.000480,0.000484,0.000475,0.000485,0.000490,0.000484,0.000486,0.000491,0.000494,0.000494,0.000489,0.000494,0.000501,0.000503,0.000500,0.000501,0.000503,0.000502,0.000501,0.000499,...,0.000695,0.000685,0.000694,0.000695,0.000674,0.000676,0.000670,0.000665,0.000682,0.000686,0.000687,0.000703,0.000710,0.000719,0.000728,0.000724,0.000722,0.000731,0.000716,0.000712,0.000715,0.000714,0.000698,0.000703,0.000708,0.000690,0.000700,0.000709,0.000698,0.000679,0.000672,0.000681,0.000686,0.000697,0.000706,0.000708,0.000715,0.000720,0.000707,0.000710
4.0,0.000887,0.000875,0.000882,0.000873,0.000861,0.000839,0.000832,0.000831,0.000851,0.000863,0.000849,0.000843,0.000840,0.000830,0.000831,0.000851,0.000846,0.000850,0.000850,0.000842,0.000830,0.0008

In [94]:
financials_df

,isin,exchange_id,sector,industry,market_cap,market_cap_group
ticker,,,,,,
FLWS,US68243Q1067,663,Consumer Cyclical,Specialty Retail,6.432e+07,5.0
TXG,US88025U1097,663,Healthcare,Health Information Services,101340000,6.0
YI,US68247Q1022,663,Healthcare,Health Information Services,82430000,6.0
FCCY,US31986N1028,663,Financial,Banks - Regional,10230000,1.0
SRCE,US3369011032,663,Financial,Banks - Regional,25550000,2.0
...,...,...,...,...,...,...
CNET,US98880R1095,663,Communication Services,Advertising Agencies,21720000,2.0
ZYME,CA98985W1023,676,Healthcare,Biotechnology,50900000,4.0
ZYNE,US98986X1090,663,Healthcare,Drug Manufacturers - Specialty & Generic,29240000,3.0


In [109]:
open_df_index = pd.read_csv(str(ele + ' open'), index_col='date')
open_df_index

,Basic Materials,Communication Services,Consumer Cyclical,Consumer Defensive,Energy,Financial,Healthcare,Industrials,Real Estate,Technology,Utilities
date,,,,,,,,,,,
2019-07-29,0.001597,0.004899,0.004560,0.003225,0.003054,0.007503,0.005344,0.003343,0.001126,0.007181,0.001193
2019-07-30,0.001586,0.004844,0.004509,0.003228,0.003029,0.007405,0.005306,0.003314,0.001125,0.007101,0.001191
2019-07-31,0.001598,0.004836,0.004510,0.003212,0.003058,0.007429,0.005324,0.003353,0.001133,0.007144,0.001189
2019-08-01,0.001555,0.004807,0.004478,0.003182,0.002991,0.007370,0.005295,0.003309,0.001129,0.007038,0.001181
2019-08-02,0.001533,0.004741,0.004363,0.003162,0.002961,0.007225,0.005281,0.003244,0.001127,0.006915,0.001197
...,...,...,...,...,...,...,...,...,...,...,...
2021-03-12,0.002160,0.007232,0.007107,0.003434,0.002512,0.008969,0.006744,0.004264,0.001169,0.011729,0.001167
2021-03-15,0.002167,0.007236,0.007183,0.003464,0.002507,0.008980,0.006780,0.004323,0.001191,0.011817,0.001179
2021-03-16,0.002170,0.007302,0.007265,0.003463,0.002467,0.008913,0.006866,0.004344,0.001208,0.012039,0.001189


In [112]:
str(financials_df[str(ele)].loc[ticker])

'Healthcare'

In [113]:
open_df_index.loc['Healthcare']
# ti dà errore perchè dovrai transporlo 

KeyError: ignored

In [101]:
open_df_bridge = open_df.transpose().loc[financials_df.index]
high_df_bridge = high_df.transpose().loc[financials_df.index]
low_df_bridge = low_df.transpose().loc[financials_df.index]
adjclose_df_bridge = adjclose_df.transpose().loc[financials_df.index]

for ele in list_of_history:
  if ele == 'price':
    indeces_object_dictionary['{0}'.format(ele)].open = open_df.transpose().loc[financials_df.index]
    indeces_object_dictionary['{0}'.format(ele)].high = high_df.transpose().loc[financials_df.index]
    indeces_object_dictionary['{0}'.format(ele)].low = low_df.transpose().loc[financials_df.index]
    indeces_object_dictionary['{0}'.format(ele)].adjclose = adjclose_df.transpose().loc[financials_df.index]
  else:
    open_df_index = pd.read_csv(str(ele + ' open'), index_col='date').transpose()
    high_df_index = pd.read_csv(str(ele + ' high'), index_col='date').transpose()
    low_df_index = pd.read_csv(str(ele + ' low'), index_col='date').transpose()
    adjclose_df_index = pd.read_csv(str(ele + ' adjclose'), index_col='date').transpose()
    
    indeces_object_dictionary['{0}'.format(ele)].open = open_df.transpose().loc[financials_df.index].copy()
    indeces_object_dictionary['{0}'.format(ele)].high = high_df.transpose().loc[financials_df.index].copy()
    indeces_object_dictionary['{0}'.format(ele)].low = low_df.transpose().loc[financials_df.index].copy()
    indeces_object_dictionary['{0}'.format(ele)].adjclose = adjclose_df.transpose().loc[financials_df.index].copy()

    for ticker in financials_df.index:
      indeces_object_dictionary['{0}'.format(ele)].open = open_df_index.loc[str(financials_df[str(ele)].loc[ticker])]
      indeces_object_dictionary['{0}'.format(ele)].high = high_df_index.loc[str(financials_df[str(ele)].loc[ticker])]
      indeces_object_dictionary['{0}'.format(ele)].low = low_df_index.loc[str(financials_df[str(ele)].loc[ticker])]
      indeces_object_dictionary['{0}'.format(ele)].adjclose = adjclose_df_index.loc[str(financials_df[str(ele)].loc[ticker])]

KeyboardInterrupt: ignored

In [86]:
np.array([col] * len(list_of_history))

array(['222-22', '222-22', '222-22', '222-22', '222-22'], dtype='<U6')

In [ ]:
# ORA DEVI METTERE TUTTI GLI ELEMENTI DEI DIZIONARI INSIEME
ticker_df_index = indeces_object_dictionary['exchange_id'].open.index
for num, col in enumerate(indeces_object_dictionary['exchange_id'].open.columns):
  
  header_list = [np.array([col] * len(list_of_history)), np.array(list_of_history)] 
  
  if num == 0:
    data_list = []
    for ele in list_of_history:
      data_list.append(indeces_object_dictionary['{0}'.format(ele)].open)
    open_trial = pd.DataFrame(data=np.column_stack((data_list)), index=indeces_object_dictionary['price')].open.index, columns=header_list)
   
    data_list = []
    for ele in list_of_history:
      data_list.append(indeces_object_dictionary['{0}'.format(ele)].high)
    high_trial = pd.DataFrame(data=np.column_stack((data_list)), index=indeces_object_dictionary['price')].high.index, columns=header_list)
    
    data_list = []
    for ele in list_of_history:
      data_list.append(indeces_object_dictionary['{0}'.format(ele)].low)
    low_trial = pd.DataFrame(data=np.column_stack((data_list)), index=indeces_object_dictionary['price')].low.index, columns=header_list)

    data_list = []
    for ele in list_of_history:
      data_list.append(indeces_object_dictionary['{0}'.format(ele)].open)
    adjclose_trial = pd.DataFrame(data=np.column_stack((data_list)), index=indeces_object_dictionary['price')].adjclose.index, columns=header_list)    

  else: 
    data_list = []
    for ele in list_of_history:
      data_list.append(indeces_object_dictionary['{0}'.format(ele)].open)
    open_trial2 = pd.DataFrame(data=np.column_stack((data_list)), index=indeces_object_dictionary['price')].open.index, columns=header_list)
    open_trial = open_trial.join(open_trial2, how='left')

    data_list = []
    for ele in list_of_history:
      data_list.append(indeces_object_dictionary['{0}'.format(ele)].high)
    high_trial2 = pd.DataFrame(data=np.column_stack((data_list)), index=indeces_object_dictionary['price')].high.index, columns=header_list)
    high_trial = high_trial.join(high_trial2, how='left')


    data_list = []
    for ele in list_of_history:
      data_list.append(indeces_object_dictionary['{0}'.format(ele)].low)
    low_trial2 = pd.DataFrame(data=np.column_stack((data_list)), index=indeces_object_dictionary['price')].low.index, columns=header_list)
    low_trial = low_trial.join(low_trial2, how='left')

    data_list = []
    for ele in list_of_history:
      data_list.append(indeces_object_dictionary['{0}'.format(ele)].open)
    adjclose_trial2 = pd.DataFrame(data=np.column_stack((data_list)), index=indeces_object_dictionary['price')].adjclose.index, columns=header_list)
    adjclose_trial = adjclose_trial.join(adjclose_trial2, how='left')

In [33]:
# ORA DEVI METTERE TUTTI GLI ELEMENTI DEI DIZIONARI INSIEME
ticker_df_index = open_df_prezzo.index
for num, col in enumerate(open_df_prezzo.columns):
  lista = [np.array([col, col]), np.array(['Price','ExchangeID'])] 
  if num == 0:
    listaDiProva = [open_df_price[col].values, open_df_priceEmpty[col].values]
    open_trial = pd.DataFrame(data=np.column_stack((listaDiProva)), index= open_df_price.index, columns=lista)
    high_trial = pd.DataFrame(data=np.column_stack((high_df_price[col].values, high_df_priceEmpty[col].values)), index= high_df_price.index, columns=lista)
    low_trial = pd.DataFrame(data=np.column_stack((low_df_price[col].values, low_df_priceEmpty[col].values)), index= low_df_price.index, columns=lista)
    adjclose_trial = pd.DataFrame(data=np.column_stack((adjclose_df_price[col].values, adjclose_df_priceEmpty[col].values)), index= adjclose_df_price.index, columns=lista)
  else: 
    listaDiProva = [open_df_price[col].values, open_df_priceEmpty[col].values]
    open_trial2 = pd.DataFrame(data=np.column_stack((listaDiProva)), index= open_df_price.index, columns=lista)
    open_trial = open_trial.join(open_trial2, how='left')
    high_trial2 = pd.DataFrame(data=np.column_stack((high_df_price[col].values, high_df_priceEmpty[col].values)), index= high_df_price.index, columns=lista)
    high_trial = high_trial.join(high_trial2, how='left')
    low_trial2 = pd.DataFrame(data=np.column_stack((low_df_price[col].values, low_df_priceEmpty[col].values)), index= low_df_price.index, columns=lista)
    low_trial = low_trial.join(low_trial2, how='left')
    adjclose_trial2 = pd.DataFrame(data=np.column_stack((adjclose_df_price[col].values, adjclose_df_priceEmpty[col].values)), index= adjclose_df_price.index, columns=lista)
    adjclose_trial = adjclose_trial.join(adjclose_trial2, how='left')

In [25]:
open_df_exchange_id_index = pd.read_csv('exchange_id open', index_col='date').transpose()
high_df_exchange_id_index = pd.read_csv('exchange_id high', index_col='date').transpose()
low_df_exchange_id_index = pd.read_csv('exchange_id low', index_col='date').transpose()
adjclose_df_exchange_id_index = pd.read_csv('exchange_id adjclose', index_col='date').transpose()

open_df_sector_index = pd.read_csv('sector open', index_col='date').transpose()
high_df_sector_index = pd.read_csv('sector high', index_col='date').transpose()
low_df_sector_index = pd.read_csv('sector low', index_col='date').transpose()
adjclose_df_sector_index = pd.read_csv('sector adjclose', index_col='date').transpose()

open_df_industry_index = pd.read_csv('industry open', index_col='date').transpose()
high_df_industry_index = pd.read_csv('industry high', index_col='date').transpose()
low_df_industry_index = pd.read_csv('industry low', index_col='date').transpose()
adjclose_df_industry_index = pd.read_csv('industry adjclose', index_col='date').transpose()

open_df_market_cap_index = pd.read_csv('market_cap open', index_col='date').transpose()
high_df_market_cap_index = pd.read_csv('market_cap high', index_col='date').transpose()
low_df_market_cap_index = pd.read_csv('market_cap low', index_col='date').transpose()
adjclose_df_market_cap_index = pd.read_csv('market_cap adjclose', index_col='date').transpose()

In [27]:
for ticker in financials_df.index:
  open_df_priceEmpty.loc[ticker] = open_df_exchange_id_index.loc[str(financials_df['exchange_id'].loc[ticker])]
  high_df_priceEmpty.loc[ticker] = high_df_exchange_id_index.loc[str(financials_df['exchange_id'].loc[ticker])]
  low_df_priceEmpty.loc[ticker] = low_df_exchange_id_index.loc[str(financials_df['exchange_id'].loc[ticker])]
  adjclose_df_priceEmpty.loc[ticker] = adjclose_df_exchange_id_index.loc[str(financials_df['exchange_id'].loc[ticker])]

[array([ 30.86000061, 166.88999939,  18.60000038, ...,   5.38999987,
         16.28000069,  10.        ]),
 array([0.02250869, 0.02250869, 0.02250869, ..., 0.02250869, 0.02250869,
        0.02250869])]

In [30]:
open_trial = open_trial.stack(list(range(open_trial.columns.nlevels)))
high_trial = high_trial.stack(list(range(high_trial.columns.nlevels)))
low_trial = low_trial.stack(list(range(low_trial.columns.nlevels)))
adjclose_trial = adjclose_trial.stack(list(range(adjclose_trial.columns.nlevels)))

In [31]:
data = {'open': open_trial, 
        'high': high_trial,
        'low': low_trial,
        'adjclose': adjclose_trial}

df_concat = pd.concat(data, axis=1)
df_concat.dropna(axis = 0, how = 'any', inplace=True)
df_concat

open       high        low   adjclose
ticker                                                                  
FLWS   2019-07-29 ExchangeID   0.014007   0.014098   0.013777   0.013731
                  Price       19.440001  19.850000  19.340000  19.680000
       2019-07-30 ExchangeID   0.013843   0.014019   0.013731   0.013687
                  Price       19.600000  20.059999  19.540001  20.030001
       2019-07-31 ExchangeID   0.013923   0.014062   0.013581   0.013527
...                                 ...        ...        ...        ...
ZNGA   2021-03-16 Price       10.060000  10.470000  10.050000  10.310000
       2021-03-17 ExchangeID   0.022528   0.023032   0.022231   0.022826
                  Price       10.160000  10.170000   9.800000  10.130000
       2021-03-18 ExchangeID   0.022509   0.022749   0.022046   0.022139
                  Price       10.000000  10.050000   9.830000   9.890000

[3528936 rows x 4 columns]

In [ ]:
%%time
for ticker in ticker_list:
    
    try:
        stock_object_dictionary['{0}'.format(ticker)].history = 
        
        # features dataframe, without classification column
        features_df = df_concat.loc[ticker]
        features_df.drop('label', axis=1, inplace=True)
        
        # classification array
        classification_array = df_concat.loc[ticker]['label'].values

        # sacaling data
        df_sc_scaled = StandardScaler().fit_transform(features_df)

        shuffle_value = True # shuffle, because of this we cannot stratify our label
        stratify_value = classification_array # stratify fashion 
        train_size = 0.80
        random_state = None

        stock_object_dictionary['{0}'.format(ticker)].confusion_matrix2, stock_object_dictionary['{0}'.format(ticker)].accuracy_report2, stock_object_dictionary['{0}'.format(ticker)].logistic_regression2 = logistic_reg(df_sc_scaled, classification_array, train_size, shuffle_value, stratify_value, random_state) 
    
    except:
        pass

### &nbsp;&nbsp;&nbsp; * SKLearn preprocessing import to scale data

In [ ]:
from sklearn import preprocessing

### &nbsp;&nbsp;&nbsp; %FEATURE% Preprocessing of input data

In [ ]:
# relative variation from open to adjusted close price
adjclose_rel_var_df = (adjclose_df-open_df)/open_df

### &nbsp;&nbsp;&nbsp; %FEATURE% Absolute variation between high and low price

In [ ]:
# absolute variation between high and low price
high_low_var_df = (high_df-low_df)

# to scale the absolute variation between min&max value
high_low_var_scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
high_low_var_scaled = high_low_var_scaler.fit_transform(high_low_var_df)
high_low_var_scaled_df = pd.DataFrame(data=high_low_var_scaled, index=high_low_var_df.index, columns=high_low_var_df.columns)

### &nbsp;&nbsp;&nbsp; %FEATURE% High low absolute variation over adjusted close price

In [ ]:
# high low absolute variation over adjusted close price
high_low_var_df_adjclose = high_low_var_df/adjclose_df

### &nbsp;&nbsp;&nbsp; %FEATURE% Log Return (adjusted close price log return)

In [ ]:
adjclose_df_log_return = np.log(adjclose_df/adjclose_df.shift(1))

### &nbsp;&nbsp;&nbsp; * Construction of stack dataset with all features and label to classify

In [ ]:
adjclose_rel_var_df_t = adjclose_rel_var_df.transpose()
adjclose_rel_var_df_stack = adjclose_rel_var_df_t.stack(dropna=False)

high_low_var_scaled_df_t = high_low_var_scaled_df.transpose()
high_low_var_scaled_df_stack = high_low_var_scaled_df_t.stack(dropna=False)

high_low_var_df_adjclose_t = high_low_var_df_adjclose.transpose()
high_low_var_df_adjclose_stack = high_low_var_df_adjclose_t.stack(dropna=False)

adjclose_df_t = adjclose_df.transpose()
adjclose_df_stack = adjclose_df_t.stack(dropna=False)

volume_df_t = volume_df.transpose()
volume_df_stack = volume_df_t.stack(dropna=False)

# classification variable 
classification_df = (adjclose_df_log_return > 0) * 1
classification_df_t = classification_df.transpose()
classification_df_t.shift(axis = 1, periods = -1) # sign the day before a rally up with 1 and vice versa with 0 
classification_df_stack = classification_df_t.shift(axis = 1, periods = -1).stack(dropna=False)

data = {'adjclose_rel_var': adjclose_rel_var_df_stack, 
        'high_low_var_scaled': high_low_var_scaled_df_stack,
        'high_low_var_adjclose': high_low_var_df_adjclose_stack,
        'adjclose': adjclose_df_stack,
        'volume': volume_df_stack,
        'label': classification_df_stack}  

df_concat = pd.concat(data, axis=1)
df_concat.dropna(axis = 0, how = 'any', inplace=True)
df_concat

### &nbsp;&nbsp;&nbsp; * Check the correct number of row in the previous dataframe

In [ ]:
# check the correct number of row in the previous dataframe
len(open_df.columns) * len(open_df.index) - len(open_df.columns)

### &nbsp;&nbsp;&nbsp; * Construction of stack dataset with other features and label to classify

In [ ]:
open_df_t = open_df.loc[adjclose_df_log_return.index[0]:adjclose_df_log_return.index[-1]].transpose()
open_df_stack = open_df_t.stack()

high_df_t = high_df.loc[adjclose_df_log_return.index[0]:adjclose_df_log_return.index[-1]].transpose()
high_df_stack = high_df_t.stack()

low_df_t = low_df.loc[adjclose_df_log_return.index[0]:adjclose_df_log_return.index[-1]].transpose()
low_df_stack = low_df_t.stack()

volume_df_t = volume_df.loc[adjclose_df_log_return.index[0]:adjclose_df_log_return.index[-1]].transpose()
volume_df_stack = volume_df_t.stack()

data = {'open': open_df_stack, 
        'high': high_df_stack,
        'low': low_df_stack,
        'adjclose': adjclose_df_stack,
        'volume': volume_df_stack,
        'label': classification_df_stack} # taken from before computation

df_concat2 = pd.concat(data, axis = 1)
df_concat2.dropna(axis = 0, how = 'any', inplace = True)
df_concat2

##### Remember: .std() is sample deviation, whereas the standardization thru StandardScaler use the standard deviation (the difference is the denominator, in the sample std it is used N-1). Moreover, to compute the standard deviation, you can do it in this way: .std(ddof=0)

# 2.b Second part: computation and assessing 

# 3. Third part: Deep Learning

In [ ]:
import tensorflow as tf 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
print(tf.__version__)

### &nbsp;&nbsp;&nbsp; %FUNCTION% Function for calling back DL training

In [ ]:
accuracy_limit=.7
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy')>accuracy_limit):
            print('--- accuracy higher than ', accuracy_limit)
            self.model.stop_training=True

In [ ]:
ticker = 'AAPL'

# features dataframe, without classification column
features_df = df_concat.loc[ticker]
features_df.drop('label', axis=1, inplace=True)

# classification array
classification_array = df_concat.loc[ticker]['label'].values

# sacaling data
df_sc_scaled = StandardScaler().fit_transform(features_df)

shuffle_value = False # no shuffle, because of this we cannot stratify our label
stratify_value = None
train_size = 0.80
random_state = None

# define train set and test set 
x_train, x_test, y_train, y_test = train_test_split(features_df, classification_array, train_size = train_size, shuffle = shuffle_value, stratify = stratify_value, random_state = random_state)

In [ ]:
callbacks=myCallback()

In [ ]:
model=tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                   tf.keras.layers.Dense(units=500, activation=tf.nn.relu),
                                   tf.keras.layers.Dense(units=2, activation=tf.nn.softmax)])

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=100, callbacks=[callbacks])

In [ ]:
stock_object_dictionary['{0}'.format(ticker)].confusion_matrix, stock_object_dictionary['{0}'.format(ticker)].accuracy_report = logistic_reg(df_sc_scaled, classification_array, train_size, shuffle_value, stratify_value, random_stat) 

### a. Trial 1: No Shuffle (i.e. No Stratify) - 80/20 - scaled features